## <b>Description</b>
MNIST ("Modified National Institute of Standards and Technology") is the de facto “hello world” dataset of computer vision. Since its release in 1999, this classic dataset of handwritten images has served as the basis for benchmarking classification algorithms. As new machine learning techniques emerge, MNIST remains a reliable resource for researchers and learners alike.**


### <b>Acknowledgements </b>
More details about the dataset, including algorithms that have been tried on it and their levels of success, can be found at http://yann.lecun.com/exdb/mnist/index.html. The dataset is made available under a Creative Commons Attribution-Share Alike 3.0 license.

## Importing Fastai for classification

In [ ]:
from fastai.vision import *

## Creating a Custom Class and Function to convert the CSV to ImageItemList

In [ ]:
class CustomImageItemList(ImageItemList):
    def open(self, fn):
        img = fn.reshape(28, 28)
        img = np.stack((img,)*3, axis=-1) # convert to 3 channels
        return Image(pil2tensor(img, dtype=np.float32))

    @classmethod
    def custom_csv(cls, path:PathOrStr, csv_name:str, imgIdx:int=1, header:str='infer', **kwargs) -> 'ItemList':
        df = pd.read_csv(Path(path)/csv_name, header=header)
        res = super().from_df(df, path=path, cols=0, **kwargs)
        # convert pixels to an ndarray
        res.items = df.iloc[:,imgIdx:].apply(lambda x: x.values / 783.0, axis=1).values
        return res

In [ ]:
test = CustomImageItemList.custom_csv(path=Path('../input/'), csv_name='test.csv', imgIdx=0)
data = (CustomImageItemList.custom_csv(path=Path('../input/'), csv_name='train.csv')
                       .random_split_by_pct(.2)
                       .label_from_df(cols='label')
                       .add_test(test, label=0)
                       .databunch(bs=64, num_workers=0)
                       .normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

## Using Resnet50 for Classifying

In [ ]:
learn = create_cnn(data, models.resnet50, metrics=accuracy, model_dir = Path('../working/'))

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(1e-2,1e-1))

In [ ]:
learn.save('50-stage-1')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2,slice(7e-6,1e-5))

In [ ]:
learn.save('50-stage-2')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,slice(2e-5,1e-4))

In [ ]:
learn.save('50-stage-3')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,slice(1e-4,2e-4))

In [ ]:
learn.save('50-stage-4')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,slice(5e-6,2e-5))

In [ ]:
learn.save('50-stage-5')

## Creating The Predictions and saving it as submissions.csv

In [ ]:
learn.load('50-stage-5')
preds, y, losses = learn.get_preds(ds_type=DatasetType.Test, with_loss=True)

In [ ]:
y = torch.argmax(preds, dim=1)

In [ ]:
preds.shape, len(y)

In [ ]:
submission_df = pd.DataFrame({'ImageId': range(1, len(y) + 1), 'Label': y}, columns=['ImageId', 'Label'])
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
!head submission.csv